<a href="https://colab.research.google.com/github/basketbob/ML/blob/main/netology_DS/%D0%B8%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D0%B0%D1%8F_python_%D0%9A%D1%83%D0%B7%D0%B8%D0%BD%D0%92%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы
df_movies  = pd.read_csv('u.item.csv', sep='|')
df_raiting = pd.read_csv('u.data.csv', sep=';')
df_movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок
df_cnt = df_raiting[['user id', 'rating']].groupby(['user id']).count().reset_index()
top_user_id = df_cnt.sort_values(by=['rating'], ascending=False).iloc[0]['user id']
print(top_user_id)

405


In [ ]:
# Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь
df_movies_by_user = df_raiting[['user id', 'item id']][df_raiting['user id'] == top_user_id]
df_movies_by_user.head()

,user id,item id
12276,405,56
12383,405,592
12430,405,1582
12449,405,171
12460,405,580


In [ ]:
# Добавьте к датафрейму из задания 3 столбцы
df_raiting = df_raiting.merge(df_movies, how='left', left_on='item id', right_on='movie id')

In [ ]:
df_raiting['count_rating'] = df_raiting['rating'].groupby(df_raiting['movie id']).transform('count')
df_raiting['sum_rating'] = df_raiting['rating'].groupby(df_raiting['movie id']).transform('sum')

df_raiting['release_year'] = pd.to_datetime(df_raiting['release date']).dt.year
df_raiting = df_raiting[[
    'user id', 'item id', 'rating', 'unknown', 'Action', 'Adventure', 'Animation',
    "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
    'Western', 'count_rating', 'sum_rating', 'release_year']]
df_raiting.head()

In [ ]:
df_movies_by_user = df_movies_by_user.merge(df_raiting, how='left', left_on='item id', right_on='item id')
df_raiting.head()

,user id,item id,rating,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,count_rating,sum_rating,release_year
0,196,242,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,117,467,1997.0
1,186,302,3,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,297,1236,1997.0
2,22,377,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,13,28,1994.0
3,244,51,2,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,81,280,1994.0
4,166,346,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,126,459,1997.0


In [ ]:
# Сформируйте X_train, X_test, y_train, y_test
X, y = df_movies_by_user[[
    'unknown', 'Action', 'Adventure', 'Animation', 'Film-Noir',
    "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
    'War', 'Western', 'count_rating', 'sum_rating', 'release_year'
    ]], df_movies_by_user['rating']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, model.predict(X_test))

1.0309138733084966

# SPARK
Загрузить данные в spark

Средствами спарка вывести среднюю оценку для каждого фильма

Посчитайте средствами спарка среднюю оценку для каждого жанра

В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)



In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar -xvf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.option("delimiter", ";").csv('u.data.csv', inferSchema=True, header=True)
df_item = spark.read.option("delimiter", "|").csv('u.item.csv', inferSchema=True, header=True)
df.groupby('item id').avg('rating').show(5)

+-------+------------------+
|item id|       avg(rating)|
+-------+------------------+
|    496| 4.121212121212121|
|    471|3.6108597285067874|
|    463| 3.859154929577465|
|    148|          3.203125|
|   1342|               2.5|
+-------+------------------+
only showing top 5 rows



In [ ]:
df_grp = df.groupby('item id').count()
df_grp.sort(df_grp['count'].desc()).show(5)

+-------+-----+
|item id|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
+-------+-----+
only showing top 5 rows



In [ ]:
df_grp.sort(df_grp['count'].asc()).show(5)

+-------+-----+
|item id|count|
+-------+-----+
|   1460|    1|
|   1618|    1|
|   1339|    1|
|   1645|    1|
|   1650|    1|
+-------+-----+
only showing top 5 rows



In [ ]:
df = df.join(df_item, df['item id'] == df_item['movie id'], 'left')
df.show(5)

+-------+-------+------+---------+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|user id|item id|rating|timestamp|movie id|         movie title|release date|video release date|            IMDb URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+-------+-------+------+---------+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|    196|    242|     3|881250949|     242|        Kolya (1996)| 24-Jan-1997|              null|http://us.imdb.co...|      0|     0|        0|        0|         0|     1|    0|        

In [ ]:
genres = ['unknown', 'Action', 'Adventure', 'Animation', 'Film-Noir',
          "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
          'War', 'Western']

genres_rating = {}

for genre in genres:
  genres_rating[genre] = df[df[genre] == 1].groupby().avg('rating').collect()
  genres_rating[genre] = genres_rating[genre][0].__getitem__('avg(rating)')

print(genres_rating)

{'unknown': 3.2, 'Action': 3.480245417953027, 'Adventure': 3.503526503308369, 'Animation': 3.5766990291262135, 'Film-Noir': 3.9215233698788228, "Children's": 3.3532442216652742, 'Comedy': 3.3940734781442745, 'Crime': 3.6322780881440098, 'Documentary': 3.6728232189973613, 'Drama': 3.6873793708484772, 'Fantasy': 3.2152366863905324, 'Horror': 3.2903893172841827, 'Musical': 3.521396851029471, 'Mystery': 3.63813155386082, 'Romance': 3.621704948358255, 'Sci-Fi': 3.5607227022780834, 'Thriller': 3.5090069495245064, 'War': 3.815811874866993, 'Western': 3.6132686084142396}
